# Домашнее задание 1. Извлечение ключевых слов

При выполнении домашнего задания можно пользоваться тетрадкой с семинара.

### Описание задания:

1. Подготовить мини-корпус (4-5 текстов или до 10 тысяч токенов) с разметкой ключевых слов.
Желательно указать источник корпуса и описать, в каком виде там были представлены ключевые слова.

2. Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.

3. Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25).

4. Оценить точность, полноту, F-меру выбранных методов относительно имеющейся разметки.

5. Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется);
предложить свои методы решения этих проблем.

### Критерии оценки:

По 2 балла на каждый пункт.

### Формат сдачи задания:

Jupyter-notebook на гитхабе (запишите адрес своего репозитория [сюда](https://forms.gle/Z4z3JsHbvM6Ghroo9))

### Дедлайн: 

18 ноября 2019 10:00мск

Следующую домашку сделаю лучше :(

Мини-корпус:

In [1]:
text = open('hist.txt', 'r')
text = text.read()

На elibrary можно посмотреть научные статьи и их ключевые слова.

Вот ключевые слова, представленные на сайте:  	
АЛЛЕГОРИЯ, ТРИУМФАЛЬНАЯ АРКА, КОРОНАЦИЯ, РЕЗИДЕНЦИЯ, ФЕЙЕРВЕРК, МАСКАРАД, ИМПЕРАТРИЦА, МАНИФЕСТ

Вот ключевые слова, которые я придумала сама: КОРОНАЦИЯ, ЦЕРЕМОНИЯ, ИМПЕРАТРИЦА, ФЕЙЕРВЕРК, ВИЗУАЛИЗАЦИЯ

In [38]:
y = ['аллегория', 'триумфальная арка', 'коронация', 'резиденция', 'фейерверк', 'маскарад', 'императрица', 'манифест']

In [31]:
preds = ['коронация', 'церимония', 'императрица', 'фейерверк', 'визуализация']

In [56]:
def metrics(p, real):
    tp = 0
    fn = 0
    fp = 0
    #tn
    for pred in p:
        if pred in real:
            tp += 1 #3
        if pred not in real:
            fp += 1 #2
    for i in real:
        if i not in p:
            fn += 1 #5
    accuracy = tp / (tp + fn + fp)
    precision = tp / (tp + fp) #3/3+2
    recall = tp / (tp + fn)
    try:
        f1 = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
    
    print('accuracy: ' + str(accuracy) + '\n' + 'precision: ' + str(precision) + '\n' + 'f1: ' + str(f1))
    return

In [54]:
metrics(preds, y)

accuracy: 0.3
precision: 0.6
f1: 0.4615384615384615


Вот ключевые слова, которые мы нашли с помощью RAKE и TextRank:

In [6]:
import RAKE

In [7]:
import nltk

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayakorotkaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords

In [10]:
stop = stopwords.words('russian')

In [11]:
rake = RAKE.Rake(stop)

In [12]:
wiki_kw_list = rake.run(text, maxWords=3, minFrequency=2)

In [57]:
preds_rake = []
for i in wiki_kw_list:
    preds_rake.append(i[0])

In [60]:
preds_rake[:10]

['славяно-греко-латинской академии',
 'время рождественского поста',
 'повышенной центральной частью',
 'коронации екатерины i',
 'фейерверк',
 'триумфальные ворота',
 'день коронации',
 'грановитой палате',
 'головинскую усадьбу',
 'соборной площади']

In [59]:
metrics(preds_rake, y)

accuracy: 0.0
precision: 0.0
f1: 0.0


In [63]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [64]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [65]:
rake.run(normalize_text(text), maxWords=3, minFrequency=2)[:10]

[('славяно-греко-латинской ̆ академия', 16.0),
 ('время рождественский пост', 8.25),
 ('коронация екатерина i', 7.225121627598408),
 ('триумфальный ворот', 4.8901098901098905),
 ('екатерина ii', 4.491228070175438),
 ('день коронация', 4.352941176470589),
 ('триумфальный арка', 4.294871794871795),
 ('успенский собор', 4.181818181818182),
 ('фея ̆ ерверка', 4.0),
 ('грановитой ̆ палата', 4.0)]

TextRank

In [17]:
from gensim.summarization import keywords as kw

In [50]:
import re
import nltk
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayakorotkaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
def preprocessing(text):
    text = str(text)
    text = re.sub(r'[0-9\ufeff]', r'', text)
    normal_text = ''
    for t in tokenizer.tokenize(text):
                t = morph.parse(t)[0]
                if t.normal_form not in russian_stopwords and t.normal_form != 'это' and t.normal_form != 'весь' and not re.match(r'[0-9A-Za-z]', t.normal_form):
                    normal_form = t.normal_form + ' '
                    normal_text += normal_form
    return normal_text

In [58]:
pr_text = preprocessing(text)

In [55]:
kw(pr_text, pos_filter=[], scores=True)

[('которыи', 0.2003328293890981),
 ('екатерина', 0.1788191391177823),
 ('год', 0.15655676413917746),
 ('свои', 0.13722997139541132),
 ('кремль', 0.13589142862147868),
 ('город', 0.12026776528463498),
 ('коронационныи', 0.11433183576977796),
 ('торжество', 0.10842102065196833),
 ('скои императрица', 0.10764540181861984),
 ('жизнь россия', 0.10743175362193547),
 ('маи', 0.10720768176964851),
 ('петр', 0.10572510792885385),
 ('женскии коронация', 0.09250327718090207),
 ('сторона', 0.08870900632447405),
 ('плата', 0.08810252145366183),
 ('император', 0.08616413345213185),
 ('мочь', 0.07885311565377498),
 ('день', 0.07721641621281528),
 ('народ', 0.07644195011441632),
 ('изображение', 0.07603602494312602),
 ('маскарад', 0.07255749983725024),
 ('триумфальныи арка фея ерверка', 0.068908728032775),
 ('ворот', 0.06863500568922684),
 ('мероприятие', 0.0674793620870864),
 ('площадь', 0.06368243401068985),
 ('успенскии собор', 0.06332496091102664),
 ('наш', 0.06278136275683922),
 ('петровскии врем

In [21]:
!pip install summa

    100% |████████████████████████████████| 61kB 981kB/s ta 0:00:01
  Running setup.py bdist_wheel for summa ... done
  Stored in directory: /Users/mayakorotkaya/Library/Caches/pip/wheels/6a/09/68/e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
from summa import keywords

In [56]:
keywords.keywords(pr_text, additional_stopwords=stop, scores=True)

[('который', 0.1969535618049802),
 ('екатерина', 0.17476652580461574),
 ('год', 0.16843704468755982),
 ('свой', 0.135128818934308),
 ('кремль', 0.13318034335086706),
 ('город', 0.12223346030988501),
 ('коронационный', 0.11167985034405437),
 ('торжество', 0.11102958250574985),
 ('ской императрица', 0.10704385363815587),
 ('император пётр', 0.09733300564581332),
 ('сторона', 0.08761468642593655),
 ('плата', 0.08369289133545392),
 ('жизнь россия ский общество', 0.07849345839282283),
 ('изображение', 0.07834713989817592),
 ('мочь', 0.07732222346437896),
 ('день', 0.07564528324305089),
 ('женский коронация особый', 0.07545611541371126),
 ('маскарад', 0.07120378432687485),
 ('вечером май', 0.06868829857900366),
 ('ф', 0.0673417939555967),
 ('мероприятие', 0.06564996906132836),
 ('наш', 0.06254947839060303),
 ('площадь', 0.06254920289366109),
 ('шествие', 0.06162432779366782),
 ('петровский время', 0.06134371219932991),
 ('публичный', 0.05845138334249473),
 ('ворот никольской улица', 0.058396